# Network Analysis

Many interesting data problems can be fruitfully thought of in terms of **networks**, consisting of `nodes` of some type and the `edges` that join them.

For instance, your Facebook friends form the `nodes` of a network whose `edges` are friendship relations. A less obvious example is the World Wide Web itself, with each web page a `node` and each hyperlink from one page to another an `edge`. The nodes can be `undirected` (connecting two nodes both ways) or `directed` (connecting two nodes one way only).

## Metrics

1. **degree centrality** - most connected nodes.
2. **betweenness centrality** - identifies nodel which frequently are on the shortest paths between pairs of other nodel. In particular, the betweenness centrality of node i is computed by adding up, for every other pair of nodes j and k, the proportion of shortest paths between node j and node k that pass through i.
3. 

In [11]:
users = [
        { "id": 0, "name": "Hero" },
        { "id": 1, "name": "Dunn" },
        { "id": 2, "name": "Sue" },
        { "id": 3, "name": "Chi" },
        { "id": 4, "name": "Thor" },
        { "id": 5, "name": "Clive" },
        { "id": 6, "name": "Hicks" },
        { "id": 7, "name": "Devin" },
        { "id": 8, "name": "Kate" },
        { "id": 9, "name": "Klein" }
]

In [12]:
friendship_pairs = [(0, 1), (0, 2), (1, 2), (1, 3), (2, 3), (3, 4),
                   (4, 5), (5, 6), (5, 7), (6, 8), (7, 8), (8, 9)]

For example, the tuple (0, 1) indicates that the data scientist with id 0 (Hero) and the data scientist with id 1 (Dunn) are friends.

<img src="images/fig01-01.png" alt="Analytics, Data Science, Machine Learning" style="width: 800px;"/>


## 1. Degree centrality - Finding Key Connectors

In [13]:
# Find friends
def find_friends(node_id, friendship_pairs):
    '''
    Returns friends of a specific person using friendship_pairs.
    
    Args:
        node_id (int): The id of a node.
        friendship_map (list): a list of (X, Y) tuples, where X and Y are nodes id.

    Returns:
        friends (list): a list of friends id.
    '''
    friends = []
    for friendship in friendship_pairs:
        if friendship[0] == node_id:
            friends.append(friendship[1])
        if friendship[1] == node_id:
            friends.append(friendship[0])
    return friends

In [14]:
find_friends(1, friendship_pairs)

[0, 2, 3]

In [15]:
# Initialize the dict with an empty list for each user id:
friendships = {user["id"]: [] for user in users}
friendships

{0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: []}

In [16]:
# And loop over the friendship pairs to populate it:
for i, j in friendship_pairs:
    friendships[i].append(j)  # Add j as a friend of user i
    friendships[j].append(i)  # Add i as a friend of user j

In [17]:
friendships

{0: [1, 2],
 1: [0, 2, 3],
 2: [0, 1, 3],
 3: [1, 2, 4],
 4: [3, 5],
 5: [4, 6, 7],
 6: [5, 8],
 7: [5, 8],
 8: [6, 7, 9],
 9: [8]}

### What's the average number of connections?

In [18]:
def number_of_friends(user, friendships):
    '''
    Returns a number friends of a specific user and friendships.
    
    Args:
        user (list): a list of dics { "id": 0, "name": "Hero" }
        friendships (dict): a list of dics {0: [1, 2]}

    Returns:
        friends_count (int): number of friends of a specified user.
    '''
    user_id = user["id"]
    #print(user_id)
    friend_ids = friendships[user_id]
    #print(friend_ids)
    friends_count = len(friend_ids)
    #print(friends_count)
    return friends_count

In [19]:
number_of_friends(users[1], friendships)

3

In [20]:
total_connections = sum(number_of_friends(user, friendships) for user in users)
total_connections

24

In [21]:
num_users = len(users)
num_users

10

In [22]:
avg_connections = total_connections / num_users
avg_connections

2.4

### Who are the most connected people?

In [23]:
# Create a list (user_id, number_of_friends)
num_friends_by_id = [(user["id"], number_of_friends(user, friendships)) for user in users]
num_friends_by_id

[(0, 2),
 (1, 3),
 (2, 3),
 (3, 3),
 (4, 2),
 (5, 3),
 (6, 2),
 (7, 2),
 (8, 3),
 (9, 1)]

In [24]:
# Compute the network metric degree centrality
num_friends_by_id.sort(                                # Sort the list
       key=lambda id_and_friends: id_and_friends[1],   # by num_friends
       reverse=True)                                   # largest to smallest

# Each pair is (user_id, num_friends):
# [(1, 3), (2, 3), (3, 3), (5, 3), (8, 3),
#  (0, 2), (4, 2), (6, 2), (7, 2), (9, 1)]
num_friends_by_id

[(1, 3),
 (2, 3),
 (3, 3),
 (5, 3),
 (8, 3),
 (0, 2),
 (4, 2),
 (6, 2),
 (7, 2),
 (9, 1)]

<img src="images/fig01-02.png" alt="Analytics, Data Science, Machine Learning" style="width: 800px;"/>


## 2. Betweenness centrality

In [25]:
from typing import NamedTuple

class User(NamedTuple):
    id: int
    name: str

users = [User(0, "Hero"), User(1, "Dunn"), User(2, "Sue"), User(3, "Chi"),
         User(4, "Thor"), User(5, "Clive"), User(6, "Hicks"),
         User(7, "Devin"), User(8, "Kate"), User(9, "Klein")]

In [26]:
friend_pairs = [(0, 1), (0, 2), (1, 2), (1, 3), (2, 3), (3, 4),
                (4, 5), (5, 6), (5, 7), (6, 8), (7, 8), (8, 9)]

In [27]:
from typing import Dict, List

# type alias for keeping track of Friendships
Friendships = Dict[int, List[int]]

friendships: Friendships = {user.id: [] for user in users}

for i, j in friend_pairs:
    friendships[i].append(j)
    friendships[j].append(i)

assert friendships[4] == [3, 5]
assert friendships[8] == [6, 7, 9]

In [28]:
from collections import deque

Path = List[int]

def shortest_paths_from(from_user_id: int,
                        friendships: Friendships) -> Dict[int, List[Path]]:
    # A dictionary from user_id to *all* shortest paths to that user.
    shortest_paths_to: Dict[int, List[Path]] = {from_user_id: [[]]}

    # A queue of (previous user, next user) that we need to check.
    # Starts out with all pairs (from_user, friend_of_from_user).
    frontier = deque((from_user_id, friend_id)
                     for friend_id in friendships[from_user_id])

    # Keep going until we empty the queue.
    while frontier:
        # Remove the pair that's next in the queue.
        prev_user_id, user_id = frontier.popleft()

        # Because of the way we're adding to the queue,
        # necessarily we already know some shortest paths to prev_user.
        paths_to_prev_user = shortest_paths_to[prev_user_id]
        new_paths_to_user = [path + [user_id] for path in paths_to_prev_user]
        
        # It's possible we already know a shortest path to user_id.
        old_paths_to_user = shortest_paths_to.get(user_id, [])

        # What's the shortest path to here that we've seen so far?
        if old_paths_to_user:
            min_path_length = len(old_paths_to_user[0])
        else:
            min_path_length = float('inf')

        # Only keep paths that aren't too long and are actually new.
        new_paths_to_user = [path
                             for path in new_paths_to_user
                             if len(path) <= min_path_length
                             and path not in old_paths_to_user]

        shortest_paths_to[user_id] = old_paths_to_user + new_paths_to_user

        # Add never-seen neighbors to the frontier.
        frontier.extend((user_id, friend_id)
                        for friend_id in friendships[user_id]
                        if friend_id not in shortest_paths_to)

    return shortest_paths_to

In [29]:
# For each from_user, for each to_user, a list of shortest paths.
shortest_paths = {user.id: shortest_paths_from(user.id, friendships)
                  for user in users} 

In [30]:
shortest_paths

{0: {0: [[]],
  1: [[1]],
  2: [[2]],
  3: [[1, 3], [2, 3]],
  4: [[1, 3, 4], [2, 3, 4]],
  5: [[1, 3, 4, 5], [2, 3, 4, 5]],
  6: [[1, 3, 4, 5, 6], [2, 3, 4, 5, 6]],
  7: [[1, 3, 4, 5, 7], [2, 3, 4, 5, 7]],
  8: [[1, 3, 4, 5, 6, 8],
   [2, 3, 4, 5, 6, 8],
   [1, 3, 4, 5, 7, 8],
   [2, 3, 4, 5, 7, 8]],
  9: [[1, 3, 4, 5, 6, 8, 9],
   [2, 3, 4, 5, 6, 8, 9],
   [1, 3, 4, 5, 7, 8, 9],
   [2, 3, 4, 5, 7, 8, 9]]},
 1: {1: [[]],
  0: [[0]],
  2: [[2]],
  3: [[3]],
  4: [[3, 4]],
  5: [[3, 4, 5]],
  6: [[3, 4, 5, 6]],
  7: [[3, 4, 5, 7]],
  8: [[3, 4, 5, 6, 8], [3, 4, 5, 7, 8]],
  9: [[3, 4, 5, 6, 8, 9], [3, 4, 5, 7, 8, 9]]},
 2: {2: [[]],
  0: [[0]],
  1: [[1]],
  3: [[3]],
  4: [[3, 4]],
  5: [[3, 4, 5]],
  6: [[3, 4, 5, 6]],
  7: [[3, 4, 5, 7]],
  8: [[3, 4, 5, 6, 8], [3, 4, 5, 7, 8]],
  9: [[3, 4, 5, 6, 8, 9], [3, 4, 5, 7, 8, 9]]},
 3: {3: [[]],
  1: [[1]],
  2: [[2]],
  4: [[4]],
  0: [[1, 0], [2, 0]],
  5: [[4, 5]],
  6: [[4, 5, 6]],
  7: [[4, 5, 7]],
  8: [[4, 5, 6, 8], [4, 5, 7, 8]],
 

In [31]:
# Now we can compute betweenness centrality

betweenness_centrality = {user.id: 0.0 for user in users}

for source in users:
    for target_id, paths in shortest_paths[source.id].items():
        if source.id < target_id:      # don't double count
            num_paths = len(paths)     # how many shortest paths?
            contrib = 1 / num_paths    # contribution to centrality
            for path in paths:
                for between_id in path:
                    if between_id not in [source.id, target_id]:
                        betweenness_centrality[between_id] += contrib

In [32]:
betweenness_centrality

{0: 0.0,
 1: 3.5,
 2: 3.5,
 3: 18.0,
 4: 20.0,
 5: 20.5,
 6: 6.0,
 7: 6.0,
 8: 8.5,
 9: 0.0}

Users 0 and 9 have centrality 0 (as neither is on any shortest path between other users), whereas 3, 4, and 5 all have high centralities (as all three lie on many shortest paths). 

<img src="images/fig01-03.png" alt="Analytics, Data Science, Machine Learning" style="width: 800px;"/>

Generally the centrality numbers aren’t that meaningful themselves. What we care about is how the numbers for each node compare to the numbers for other nodes.